In [ ]:
import numpy as np
from scipy.stats import gaussian_kde

from scipy.spatial import cKDTree

import cma

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
test_range = np.arange(0,1,0.1)
# points to observe
pointsX = np.dstack((test_range, np.sin(test_range)))[0]
# the context points
pointsY = np.dstack((test_range, np.cos(test_range)))[0]
orig_shape = pointsX.shape[1]

In [ ]:
# grid
grid_spacing = 0.01
min_grid_x = min(np.min(pointsX[:,0]), np.min(pointsY[:,0]))
min_grid_y = min(np.min(pointsX[:,1]), np.min(pointsY[:,1]))
max_grid_x = max(np.max(pointsX[:,0]), np.max(pointsY[:,0]))
max_grid_y = max(np.max(pointsX[:,1]), np.max(pointsY[:,1]))

if orig_shape > 2:
    min_grid_z = min(np.min(pointsX[:,2]), np.min(pointsY[:,2]))
    max_grid_z = max(np.max(pointsX[:,2]), np.min(pointsY[:,2]))
else:
    min_grid_z = 0.0
    max_grid_z = grid_spacing

xs = np.arange(min_grid_x,max_grid_x, grid_spacing)
ys = np.arange(min_grid_y, max_grid_y, grid_spacing)
zs = np.arange(min_grid_z, max_grid_z, grid_spacing)
x, y, z = np.meshgrid(xs,ys,zs)
sampled_points = np.dstack((x,y,z)).reshape(-1,3)
    
print("grid dimensions x:({},{}) y:({},{}) z:({},{})".format(min_grid_x, max_grid_x, 
                                                   min_grid_y, max_grid_y, min_grid_z, 
                                                   max_grid_z if max_grid_z != grid_spacing else 0.0))

In [ ]:
# pad points to 3D, distance tree over Y (context)
if pointsX.shape[1] == 2:
    pointsX = np.hstack((pointsX, np.zeros((pointsX.shape[0], 1))))
if pointsY.shape[1] == 2:
    pointsY = np.hstack((pointsY, np.zeros((pointsY.shape[0], 1))))
tree = cKDTree(pointsY)

In [ ]:
# distances of grid points to the context Y
sampled_distances = tree.query(sampled_points)[0]

In [ ]:
# calculate bin width and build grid
no_of_bins = 1000
bin_width = (np.max(sampled_distances) - np.min(sampled_distances)) / no_of_bins
qd_distance_grid = np.arange(0,no_of_bins) * bin_width
print("bin width:", bin_width)

In [ ]:
# calculate silvermann factor for first gaussian
q1 = np.percentile(sampled_distances, 25)
q3 = np.percentile(sampled_distances, 75)

sil_bandwith = 0.9 * min(np.std(sampled_distances), (q3 - q1) / 1.34) * np.power(len(sampled_distances), -0.2)
print("silverman:", sil_bandwith)

In [ ]:
# build gaussian kernel from sampled context distances (qd) on grid
kde_context = gaussian_kde(sampled_distances, bw_method=sil_bandwith)
qd_pdf = kde_context(qd_distance_grid)

area = 0
i = 0
while i < len(qd_distance_grid)-1:
    area += 0.5*(qd_pdf[i]+qd_pdf[i+1])*bin_width
    i += 1
    
qd_pdf = qd_pdf/area

In [ ]:
# distances between groups of points
distances_XtoY = tree.query(pointsX)[0]

In [ ]:
# calculate silvermann factor for second gaussian
q1 = np.percentile(distances_XtoY, 25)
q3 = np.percentile(distances_XtoY, 75)

sil_bandwith = 0.9 * min(np.std(distances_XtoY), (q3 - q1) / 1.34) * np.power(len(distances_XtoY), -0.2)
print("silverman:", sil_bandwith)

In [ ]:
# build gaussian kernel from distances between X and Y on grid 
kde_XtoY = gaussian_kde(distances_XtoY, bw_method=sil_bandwith)
distances_XtoY_pdf = kde_XtoY(qd_distance_grid)

area = 0
i = 0
while i < len(qd_distance_grid)-1:
    area += 0.5*(distances_XtoY_pdf[i]+distances_XtoY_pdf[i+1])*bin_width
    i += 1
    
distances_XtoY_pdf = distances_XtoY_pdf/area

In [ ]:
# plot distances X to Y and distances Y to grid
fig, ax = plt.subplots(figsize=(12,6))
ax.plot(qd_distance_grid, distances_XtoY_pdf, label="distances X to Y")
ax.plot(qd_distance_grid, qd_pdf, label="distances Y to grid")
ax.legend()

In [ ]:
# example potential
def plummer_potential(di, threshold, sigma):
    z = (di - threshold) / np.abs(sigma)
    if z > 0:
        return -1 * np.power(1 + z * z, -0.5)
    else:
        return -1

# generic fucntion to calculate potential, gibs potential and summed potential for a vector of distances
def calculate_potential(distances, potential_func, epsilon, sigma, threshold=0):
    potential = np.zeros(len(distances))
    sum_potential = 0
    gibs_potential = np.zeros_like(potential)
    i = 0
    
    for d in distances:
        potential[i] = epsilon * potential_func(d, threshold, sigma)
        gibs_potential[i] = np.exp(-1 * potential[i])
        sum_potential += potential[i]
        
        i += 1
        
    return potential, gibs_potential, sum_potential

In [ ]:
# another example potential
def hernquist_potential(di, threshold, sigma):
    z = (di - threshold) / np.abs(sigma);
    if z > 0:
        return -1 / (1 + z)
    else:
        return -1 * (1 - z)

In [ ]:
# normalization
def calculate_Z(gibbs_potential):
    support = gibbs_potential * qd_pdf
    
    Z = 0
    
    for i,x in enumerate(support[:-1]):
        Z += (x + support[i + 1]) / 2 * bin_width
    
    return Z

# check if value is feasible (epsilon larger than 0.0 and sigma in range)
def is_feasible(x):
    if (np.abs(x[0]) > machine_epsilon) & (x[1] >= low_range) & (x[1] <= high_range):
        return True
        
    return False

In [ ]:
# returns a score given epsilon, sigma and a potential
def fit_function(params, potential):
    _, gibbs_potential, _ = calculate_potential(qd_distance_grid, potential, *params)
    Z = calculate_Z(gibbs_potential)
    
    observed_model_fit_pd_pdf = gibbs_potential * qd_pdf * (1 / Z);
    
    value = 0
    i = 0
    
    for x in observed_model_fit_pd_pdf:
        value += np.power((x - distances_XtoY_pdf[i]), 2);
        
        i += 1
    
    return value

In [ ]:
# optimizer setup
machine_epsilon = np.finfo(np.double).eps
low_range = max((min(np.min(qd_distance_grid), np.min(distances_XtoY)), machine_epsilon))
high_range = max(np.max(qd_distance_grid), np.max(distances_XtoY))

In [ ]:
# initial population. Choose a potential!
x0 = np.random.random() * 5
x1 = np.random.random() * np.mean(distances_XtoY)
cur_potential = plummer_potential

In [ ]:
# optimzer init
es = cma.CMAEvolutionStrategy([x0, x1], x1/3, 
                              inopts={"tolfun": 1e-15, "tolfunhist": 1e-13})

In [ ]:
# optimization
while not es.stop():
    # ask for a population
    X = es.ask()
    
    i = 0
    for x in X:
        # Are all values in the population feasible? If not, recalculate.
        while not is_feasible(x):
            x = es.ask(1)[0]
            
        X[i] = x
        
        i += 1
    
    # get score of current epsilon and sigma
    fit = [fit_function(x, cur_potential) for x in X]
    
    # return score to optimizer
    es.tell(X, fit)
    es.disp()
es.result_pretty()

In [ ]:
# get best values for epsilon and sigma (scale)
eps, sigma = es.best.get()[0]

In [ ]:
# plot current potential with best epsilon and sigma (scale)
fig, ax = plt.subplots(figsize=(12,6))
ax.plot(qd_distance_grid, calculate_potential(qd_distance_grid, cur_potential, eps, sigma)[0])

In [ ]:
# calculate current model with potential, best epsilon and best sigma (scale)
_,gibbs_potential,_ = calculate_potential(qd_distance_grid, cur_potential, eps, sigma)
Z = calculate_Z(gibbs_potential)
observed_model_fit_pd_pdf = gibbs_potential * qd_pdf * (1 / Z);

area = 0
i = 0
while i < len(qd_distance_grid)-1:
    area += 0.5*(observed_model_fit_pd_pdf[i]+observed_model_fit_pd_pdf[i+1])*bin_width
    i += 1
    
observed_model_fit_pd_pdf = observed_model_fit_pd_pdf/area

In [ ]:
# plot fitted model against original distances X to Y
fig, ax = plt.subplots(figsize=(12,6))
ax.plot(qd_distance_grid, distances_XtoY_pdf, label="distances X to Y")
ax.plot(qd_distance_grid, qd_pdf, label="distances Y to grid")
ax.plot(qd_distance_grid, observed_model_fit_pd_pdf, label="fitted model")
ax.legend()